# Stellar Map Builder

**What this notebook shows**
- End-to-end exploratory analysis (loading, cleaning, EDA)
- Clear visual storytelling and interpretation

**Data**
- Local files: bsc5.csv


## Step 1: Load CSV into Pandas DataFrame

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Load from the notebook folder if present; otherwise fall back to repo-relative path
csv_path = Path('bsc5.csv')
if not csv_path.exists():
    csv_path = Path('bsc5.csv')

df = pd.read_csv(csv_path)

print(f"Total number of stars: {len(df)}")

## Sanity Checks

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.describe()

In [ ]:
df.info()

## Step 3: Convert RA and Dec to Decimal Degrees

In [ ]:
import numpy as np
import pandas as pd

for col in ['RAh','RAm','RAs','DecD','DecM','DecS']:
    df[col] = pd.to_numeric(df[col], errors='coerce')


df['ra_deg'] = 15.0 * (df['RAh'].fillna(0) +
                       df['RAm'].fillna(0)/60.0 +
                       df['RAs'].fillna(0)/3600.0)

# Avoid Pandas replace downcasting warnings by using map + fillna
sign = df['DecSign'].fillna('+').map({'-': -1, '+': 1}).fillna(1).astype(int)
df['dec_deg'] = sign * (
    df['DecD'].fillna(0) +
    df['DecM'].fillna(0)/60.0 +
    df['DecS'].fillna(0)/3600.0
)

df['ra_rad']  = np.deg2rad(df['ra_deg'])
df['dec_rad'] = np.deg2rad(df['dec_deg'])

df['ra_rad'] = np.remainder(df['ra_rad'] + 2*np.pi, 2*np.pi)
df.loc[df['ra_rad'] > np.pi, 'ra_rad'] -= 2*np.pi

df['glon_rad'] = np.deg2rad(pd.to_numeric(df['GLON'], errors='coerce'))
df['glat_rad'] = np.deg2rad(pd.to_numeric(df['GLAT'], errors='coerce'))

df['glon_rad'] = np.remainder(df['glon_rad'] + 2*np.pi, 2*np.pi)
df.loc[df['glon_rad'] > np.pi, 'glon_rad'] -= 2*np.pi

## Step 4: Assign Colors by Spectral Type

In [ ]:
def color_for_sptype(sptype):
    if isinstance(sptype, str):
        if sptype.startswith('O'): return 'darkviolet'
        elif sptype.startswith('B'): return 'blue'
        elif sptype.startswith('A'): return 'deepskyblue'
        elif sptype.startswith('F'): return 'green'
        elif sptype.startswith('G'): return 'yellow'
        elif sptype.startswith('K'): return 'orange'
        elif sptype.startswith('M'): return 'red'
    return 'black'

df['color'] = df['SpType'].apply(color_for_sptype)

## Step 5: Plot the Sky Map

In [ ]:
# Plot the real star positions using computed RA/Dec (Aitoff projection)
mask = np.isfinite(df['ra_rad']) & np.isfinite(df['dec_rad'])
plt.figure(figsize=(16, 8.4))
plt.subplot(projection='aitoff')
plt.title('Map of the Sky')
plt.scatter(df.loc[mask, 'ra_rad'], df.loc[mask, 'dec_rad'], s=1, alpha=0.4, c='r')
plt.grid(True)
plt.savefig('sky_map.pdf', bbox_inches='tight')
plt.show()

## Step 5a: All-Sky Map in Equatorial Coordinates

In [ ]:
import matplotlib.patches as mpatches

df['ra_rad'] = np.remainder(df['ra_rad'] + 2*np.pi, 2*np.pi)
df.loc[df['ra_rad'] > np.pi, 'ra_rad'] -= 2*np.pi
mask = np.isfinite(df['ra_rad']) & np.isfinite(df['dec_rad'])

plt.figure(figsize=(12, 6))
plt.subplot(111, projection='aitoff')

plt.title("All-Sky Map in Equatorial Coordinates", fontsize=14, pad=20)
plt.scatter(df.loc[mask, 'ra_rad'], df.loc[mask, 'dec_rad'],
            s=2, c=df.loc[mask, 'color'], alpha=0.6)

plt.grid(True)
plt.xlabel("Right Ascension")
plt.ylabel("Declination")

legend_items = [
    mpatches.Patch(color='darkviolet', label='O Spectra'),
    mpatches.Patch(color='blue',        label='B Spectra'),
    mpatches.Patch(color='deepskyblue', label='A Spectra'),
    mpatches.Patch(color='green',       label='F Spectra'),
    mpatches.Patch(color='yellow',      label='G Spectra'),
    mpatches.Patch(color='orange',      label='K Spectra'),
    mpatches.Patch(color='red',         label='M Spectra')
]
plt.legend(handles=legend_items,
           loc='upper right', bbox_to_anchor=(0.98, 0.98),
           fontsize='small', frameon=True, framealpha=0.9)

plt.tight_layout()
plt.savefig("sky_map_equatorial.pdf", bbox_inches="tight")
plt.show()

## Step 5b: Create All-Sky Map in Galactic Coordinates

In [ ]:
# Compute galactic radians if earlier step hasn't done it
if 'glon_rad' not in df.columns or 'glat_rad' not in df.columns:
    df['glon_rad'] = np.deg2rad(pd.to_numeric(df['GLON'], errors='coerce'))
    df['glat_rad'] = np.deg2rad(pd.to_numeric(df['GLAT'], errors='coerce'))

# Normalize longitude into [-pi, pi] for Aitoff projection
df['glon_rad'] = np.remainder(df['glon_rad'] + 2*np.pi, 2*np.pi)
df.loc[df['glon_rad'] > np.pi, 'glon_rad'] -= 2*np.pi
mask = np.isfinite(df['glon_rad']) & np.isfinite(df['glat_rad'])

plt.figure(figsize=(12, 6))
plt.subplot(111, projection='aitoff')

plt.title("All-Sky Map in Galactic Coordinates", fontsize=14, pad=20)
plt.scatter(df.loc[mask, 'glon_rad'], df.loc[mask, 'glat_rad'],
            s=2, c=df.loc[mask, 'color'], alpha=0.6)

plt.grid(True)
plt.xlabel("Galactic Longitude")
plt.ylabel("Galactic Latitude")

if 'legend_items' not in globals():
    import matplotlib.patches as mpatches
    legend_items = [
        mpatches.Patch(color='darkviolet', label='O Spectra'),
        mpatches.Patch(color='blue',        label='B Spectra'),
        mpatches.Patch(color='deepskyblue', label='A Spectra'),
        mpatches.Patch(color='green',       label='F Spectra'),
        mpatches.Patch(color='yellow',      label='G Spectra'),
        mpatches.Patch(color='orange',      label='K Spectra'),
        mpatches.Patch(color='red',         label='M Spectra')
    ]
plt.legend(handles=legend_items,
           loc='upper right', bbox_to_anchor=(0.98, 0.98),
           fontsize='small', frameon=True, framealpha=0.9)

plt.tight_layout()
plt.savefig("sky_map_galactic.pdf", bbox_inches="tight")
plt.show()

Notice the difference in the view of the sky in the two coordinate systems.

The Equatorial Map shows the sky using Earth’s view. It’s lined up with the Earth’s equator, so the Milky Way looks like a skewed group across the sky. On the other hand, the Galactic Map shows the sky using the Milky Way’s view. It’s lined up with our Galaxy, so the Milky Way runs straight across the middle. But both maps have the same stars. It is just that they just use different ways to draw the sky.

## Outputs
- [sky_map.pdf](sky_map.pdf)
- [sky_map_equatorial.pdf](sky_map_equatorial.pdf)
- [sky_map_galactic.pdf](sky_map_galactic.pdf)